In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import requests
from bs4 import BeautifulSoup
import re
import time
import json
import pandas as pd
import numpy as np
import math

In [2]:
pd.set_option('display.width',4000)
pd.set_option('display.max_colwidth', 4000)
pd.set_option('display.max_columns', 3000)

In [3]:
save_path = 'C:/Users/huang/Dropbox/Travel/Shenandoah/'

In [4]:
trails_required = 50
trail_url = 'https://www.alltrails.com/parks/us/virginia/shenandoah-national-park'

In [5]:
def get_trail_link(trail_url, trails_required):
    driver = webdriver.Chrome('C:/Users/huang/Downloads/chromedriver.exe')
    
    driver.get(trail_url)
    
    # find the 'Load More Trails button'
    button = driver.find_element_by_xpath("""//*[@id="stick-bar-parent"]/div[2]/div[2]/div/button""")
    
    
    count = math.ceil(trails_required/10)
    
    # each click loads 10 trails
    # click the load more trails #times
    while count > 1:
        try:
            button.click()
            count -= 1
            time.sleep(1)
            driver.execute_script("window.stop();")
        except NoSuchElementException:
            time.sleep(2)
            driver.execute_script("window.stop();")
    
    # save urls to a list
    url_list = []
    for num in range(1,trails_required):
        elems = driver.find_element_by_xpath(f"""//*[@id="stick-bar-parent"]/div[2]/div[1]/div[2]/div[{num}]/a""")
        url_list.append(elems.get_attribute('href'))
    return url_list

In [6]:
url_list = get_trail_link(trail_url, trails_required)

In [7]:
col_name = 'name'
col_popularity = 'popularity'
col_type = 'type'
col_length = 'length'
col_difficulty = 'difficulty_rating'
col_route_type = 'route_type'
col_avg_rating = 'avg_rating'
col_features = 'features'
col_activities = 'activities'
col_num_reviews = 'num_reviews'
col_duration_minutes = 'duration_minutes'
col_link = 'link'
col_num_photos = 'num_photos'
col_profile_photo = 'profile_photo_url'
col_direction = 'direction'
col_list = [col_name, col_avg_rating, col_popularity, col_difficulty, col_length, col_duration_minutes, col_num_reviews, col_num_photos,
           col_route_type, col_features, col_activities, col_type, col_link,col_direction]

In [8]:
test_soup = BeautifulSoup(requests.get('https://www.alltrails.com/trail/us/virginia/sugarloaf-loop-via-appalachian-trail').text, 'html.parser')

In [9]:
def convert2df(url_link):
    response = requests.get(url_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for div in soup.find_all('div',{'data-react-class':"TrailDetailsCard"}):
            sub_text = div['data-react-props']
    #get json dictionary
    trail_dict = json.loads(sub_text)['trail']
    #convert to dataframe
    df_sub = pd.DataFrame.from_dict(trail_dict, orient='index').T
    df_sub[col_link] = url_link
    
    # add direction google map
    for v in soup.find_all('li',{'title': 'Get directions to the trailhead'}):
        direction_map = json.loads(v.find('div')['data-react-props'])['href']
    
    df_sub[col_direction] = 'https:'+direction_map
    print(f'{url_link} Done')

    return df_sub
    

In [10]:
df_concat = pd.DataFrame()
for link in url_list:
    df_concat = pd.concat([df_concat, convert2df(link)], sort=False, axis=0)

https://www.alltrails.com/trail/us/virginia/old-rag-mountain-loop-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/white-oak-canyon-and-cedar-run-trails?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/dark-hollow-falls-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/hawksbill-gap-loop?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/rose-river-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/bearfence-mountain-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/south-river-falls-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/stony-man-via-appalachian-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/little-stony-man-loop-via-appalachian-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/white-oak-canyon-trail?ref=result-card Done
https://www.alltrails.com/trail/us/virginia/marys-rock-via-appalachian-trail

In [11]:
df_concat.head(2)

,objectID,ID,slug,name,popularity,type,_geoloc,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,area_id,area_name,country_id,country_name,city_id,city_name,state_id,state_name,verified_map_id,features,activities,feature_names,activity_names,overview,num_reviews,num_photos,units,area_slug,abbrev_desc_i,abbrev_desc_m,duration_minutes,trail_id,city_url,park_slug,profile_photo_url,link,direction
0,trail-10234159,10234159,us/virginia/old-rag-mountain-loop-trail,Old Rag Mountain Loop,76.8127,trail,"{'lat': 38.5714, 'lng': -78.29314}",16415.3,807.72,5,L,4,5,10106831,Shenandoah National Park,313,United States of America,8044,Syria,47,Virginia,49178189,"[dogs-no, forest, river, views, wild-flowers, wildlife]",[hiking],"[Dogs No, Forest, River, Views, Wild Flowers, Wildlife]",[Hiking],None,3062,2360,i,parks/us/virginia/shenandoah-national-park,"NOTE: Entrance fees will be collected at the boundary trailheads, unless you have a current pass, and parking will be limited to available parking spots in designated parking areas only. Vehicles park","NOTE: Entrance fees will be collected at the boundary trailheads, unless you have a current pass, and parking will be limited to available parking spots in designated parking areas only. Vehicles park",305,10234159,us/virginia/syria,us/virginia/shenandoah-national-park,https://cdn-assets.alltrails.com/uploads/photo/image/19863034/large_183f600bfb84debcf39671fd017a5a52.jpg,https://www.alltrails.com/trail/us/virginia/old-rag-mountain-loop-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.5714,-78.29314"
0,trail-10112236,10112236,us/virginia/white-oak-canyon-and-cedar-run-trails,White Oak Canyon and Cedar Run Trails Loop,45.0866,trail,"{'lat': 38.53906, 'lng': -78.34808}",14484.1,722.986,5,L,3,4.5,10106831,Shenandoah National Park,313,United States of America,8044,Syria,47,Virginia,30257945,"[dogs-leash, forest, river, views, waterfall, wild-flowers, wildlife]","[hiking, nature-trips]","[Dogs Leash, Forest, River, Views, Waterfall, Wild Flowers, Wildlife]","[Hiking, Nature Trips]",None,782,1793,i,parks/us/virginia/shenandoah-national-park,The 1st section of the trail takes you to the right along about 30 big and small waterfalls! This is the strenuous section of the trip. There is a great rock wall for climbers on this section approxim,The 1st section of the trail takes you to the right along about 30 big and small waterfalls! This is the strenuous section of the trip. There is a great rock wall for climbers on this section approxim,314,10112236,us/virginia/syria,us/virginia/shenandoah-national-park,https://cdn-assets.alltrails.com/uploads/photo/image/23240147/large_76a7c339e804ea3fc6ca661cea824835.jpg,https://www.alltrails.com/trail/us/virginia/white-oak-canyon-and-cedar-run-trails?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53906,-78.34808"


In [12]:
df_final = df_concat[col_list].copy()

df_final[col_length] = (df_final[col_length]*0.000621371).map(lambda x: round(x,1))

df_final[col_duration_minutes] = df_final[col_duration_minutes].map(lambda x: f'{int(x//60)}hr {int(x%60)}mins')

In [13]:
df_final

,name,avg_rating,popularity,difficulty_rating,length,duration_minutes,num_reviews,num_photos,route_type,features,activities,type,link,direction
0,Old Rag Mountain Loop,5,76.8127,5,10.2,5hr 5mins,3062,2360,L,"[dogs-no, forest, river, views, wild-flowers, wildlife]",[hiking],trail,https://www.alltrails.com/trail/us/virginia/old-rag-mountain-loop-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.5714,-78.29314"
0,White Oak Canyon and Cedar Run Trails Loop,4.5,45.0866,5,9.0,5hr 14mins,782,1793,L,"[dogs-leash, forest, river, views, waterfall, wild-flowers, wildlife]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/white-oak-canyon-and-cedar-run-trails?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53906,-78.34808"
0,Dark Hollow Falls Trail,4.5,42.3311,3,1.4,0hr 57mins,1076,846,O,"[dogs-no, forest, kids, river, views, waterfall, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/dark-hollow-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.51971,-78.43083"
0,Hawksbill Gap Loop via Appalachian Trail,4.5,41.4116,3,2.6,1hr 33mins,708,1070,L,"[dogs, forest, kids, views, wild-flowers, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/hawksbill-gap-loop?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.55647,-78.38683"
0,Rose River Trail,4.5,40.3936,3,3.5,1hr 59mins,781,1258,L,"[beach, dogs-leash, forest, kids, river, waterfall, wild-flowers, wildlife]","[hiking, nature-trips, trail-running]",trail,https://www.alltrails.com/trail/us/virginia/rose-river-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53317,-78.42117"
0,Bearfence Mountain Trail,4.5,37.3653,3,1.0,0hr 34mins,748,540,L,"[dogs-no, forest, views, wild-flowers, wildlife]","[birding, hiking, nature-trips, fly-fishing]",trail,https://www.alltrails.com/trail/us/virginia/bearfence-mountain-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.45239,-78.4669"
0,South River Falls Trail,4.5,36.0717,3,4.6,2hr 43mins,696,718,O,"[dogs-leash, forest, kids, river, views, waterfall]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/south-river-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.38115,-78.51698"
0,Stony Man via Appalachian Trail,4.5,35.82,1,1.5,0hr 49mins,734,445,L,"[dogs-no, forest, kids, views, wild-flowers]","[birding, hiking, nature-trips, trail-running, walking]",trail,https://www.alltrails.com/trail/us/virginia/stony-man-via-appalachian-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.59318,-78.37599"
0,Little Stony Man Loop via Appalachian Trail,4.5,34.4322,3,3.4,1hr 57mins,456,1047,L,"[beach, dogs-no, forest, kids, views, wild-flowers, wildlife]","[birding, hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/little-stony-man-loop-via-appalachian-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.59308,-78.37563"
0,White Oak Canyon Trail,4.5,33.8864,5,9.5,5hr 27mins,445,1358,O,"[beach, dogs-leash, views, waterfall, wild-flowers, wildlife]","[backpacking, camping, hiking, nature-trips, trail-running, horseback-riding]",trail,https://www.alltrails.com/trail/us/virginia/white-oak-canyon-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.58589,-78.38277"


In [14]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [15]:
from IPython.core.display import HTML

df_final_html = df_final.copy()
df_final_html[col_profile_photo] = df_final_html[col_profile_photo].map(lambda x: path_to_image_html(x))

df_final_html.to_html('C:/Users/huang/Dropbox/Travel/Shenandoah/Shenandoah_trail_link.html', escape=False)

# NPS PDF MAP 

In [16]:
nps_link = 'https://www.nps.gov/shen/planyourvisit/downloadable-guides.htm#CP_JUMP_5709005'
nps_response = requests.get(nps_link)
nps_soup = BeautifulSoup(nps_response.text, 'html.parser')

In [17]:
nps_test_soup = BeautifulSoup(requests.get('https://www.nps.gov/shen/planyourvisit/downloadable-guides.htm#CP_JUMP_5709055').text, 'html.parser')

In [18]:
area_text_list = []
area_text_dict = {}
col_area_text = 'text'
for div in nps_soup.find_all('div',{'class':"Component text-content-size text-content-style ArticleTextGroup clearfix"}):
    if div.text.lower().find('accessible pdf') != -1:
        area_text = div.text.replace('\n','').lower()
        area_text_list.append(area_text)
        area_name = re.search('^(.*area) map', area_text).group(1).strip()
        area_text_dict[area_name] = {col_area_text: area_text}

In [19]:
area_text_dict

{'dickey ridge area': {'text': 'dickey ridge area map (accessible pdf - 494kb)the dickey ridge area includes the dickey ridge visitor center just five miles from the front royal entrance station in the north district of the park. this guide includes hiking information for the fox hollow trail and the snead farm loop.'},
 'compton gap area': {'text': 'compton gap area map (accessible pdf - 630kb)this guide for the compton gap area in the north district includes information about hiking the compton peaks (west and east), fort windham rocks, and lands run falls. the compton peak hike includes a unique columnar jointing geological feature.'},
 'mathews arm area': {'text': 'mathews arm area map (accessible pdf - 681kb)this guide to the mathews arm area includes hiking information on overall run falls, and the traces trail that circumnavigates the mathews arm campground.'},
 'keyser run area map (accessible pdf - 660kb)the keyser run area': {'text': 'keyser run area map (accessible pdf - 660

In [20]:
nps_trail_pdf = []
nps_prefix = 'https://www.nps.gov'
nps_dict = {}
for a in nps_soup.find_all('a', href= True):
    pdf_file = a['href']
    if pdf_file.find('Trail') != -1 and pdf_file.endswith('.pdf'):
        trail_link = nps_prefix + pdf_file
        trail_name = re.search('upload/(.*)_', trail_link).group(1)
        with open(save_path + trail_name + '.pdf', 'wb') as f:
            f.write(requests.get(trail_link).content)
        print(f'Download {trail_link}')
        nps_trail_pdf.append(trail_link)
        nps_dict[trail_name] = trail_link

Download https://www.nps.gov/shen/planyourvisit/upload/DickeyRidge_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/ComptonGap_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/MathewsArm_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/KeyserRun_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/ThorntonGap_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/Skyland_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/WhiteoakCanyon_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/OldRag_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/Hawksbill_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/BigMeadows_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/RapidanCamp_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourvisit/upload/Bearfence_RoadTrail.pdf
Download https://www.nps.gov/shen/planyourv

In [21]:
for key, value in zip(area_text_dict.keys(), nps_trail_pdf):
    area_text_dict[key][col_link] = value

In [22]:
col_area_name = 'area_map'

In [23]:
def map_pdf_link(value):
    split_str = value.lower().split(' ')
    try:
        abbr_value = split_str[0] + ' ' + split_str[1]
    except IndexError:
        abbr_value = split_str[0]
    abbr_value_0 = split_str[0]
    
    for area, detail in area_text_dict.items():
        text = detail[col_area_text]
        link = detail[col_link]
        if text.find(abbr_value) != -1:
            area_map = area
            link_map = link
            break
        elif text.find(abbr_value_0) != -1 and abbr_value_0 not in ['little']:
            area_map = area
            link_map = link
            break           
        else:
            area_map = np.nan
            link_map = np.nan
    return area_map, link_map

In [24]:
col_pdf = 'map_pdf'
col_map_name = 'nps_name'

In [25]:
df_final[col_area_name] = df_final[col_name].map(lambda x: map_pdf_link(x)[0])
df_final[col_pdf] = df_final[col_name].map(lambda x: map_pdf_link(x)[1])

In [26]:
df_final

,name,avg_rating,popularity,difficulty_rating,length,duration_minutes,num_reviews,num_photos,route_type,features,activities,type,link,direction,area_map,map_pdf
0,Old Rag Mountain Loop,5,76.8127,5,10.2,5hr 5mins,3062,2360,L,"[dogs-no, forest, river, views, wild-flowers, wildlife]",[hiking],trail,https://www.alltrails.com/trail/us/virginia/old-rag-mountain-loop-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.5714,-78.29314",old rag area,https://www.nps.gov/shen/planyourvisit/upload/OldRag_RoadTrail.pdf
0,White Oak Canyon and Cedar Run Trails Loop,4.5,45.0866,5,9.0,5hr 14mins,782,1793,L,"[dogs-leash, forest, river, views, waterfall, wild-flowers, wildlife]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/white-oak-canyon-and-cedar-run-trails?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53906,-78.34808",whiteoak canyon area,https://www.nps.gov/shen/planyourvisit/upload/WhiteoakCanyon_RoadTrail.pdf
0,Dark Hollow Falls Trail,4.5,42.3311,3,1.4,0hr 57mins,1076,846,O,"[dogs-no, forest, kids, river, views, waterfall, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/dark-hollow-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.51971,-78.43083",big meadows area,https://www.nps.gov/shen/planyourvisit/upload/BigMeadows_RoadTrail.pdf
0,Hawksbill Gap Loop via Appalachian Trail,4.5,41.4116,3,2.6,1hr 33mins,708,1070,L,"[dogs, forest, kids, views, wild-flowers, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/hawksbill-gap-loop?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.55647,-78.38683",hawksbill area,https://www.nps.gov/shen/planyourvisit/upload/Hawksbill_RoadTrail.pdf
0,Rose River Trail,4.5,40.3936,3,3.5,1hr 59mins,781,1258,L,"[beach, dogs-leash, forest, kids, river, waterfall, wild-flowers, wildlife]","[hiking, nature-trips, trail-running]",trail,https://www.alltrails.com/trail/us/virginia/rose-river-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53317,-78.42117",big meadows area,https://www.nps.gov/shen/planyourvisit/upload/BigMeadows_RoadTrail.pdf
0,Bearfence Mountain Trail,4.5,37.3653,3,1.0,0hr 34mins,748,540,L,"[dogs-no, forest, views, wild-flowers, wildlife]","[birding, hiking, nature-trips, fly-fishing]",trail,https://www.alltrails.com/trail/us/virginia/bearfence-mountain-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.45239,-78.4669",bearfence area,https://www.nps.gov/shen/planyourvisit/upload/Bearfence_RoadTrail.pdf
0,South River Falls Trail,4.5,36.0717,3,4.6,2hr 43mins,696,718,O,"[dogs-leash, forest, kids, river, views, waterfall]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/south-river-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.38115,-78.51698",south river area,https://www.nps.gov/shen/planyourvisit/upload/SouthRiver_RoadTrail.pdf
0,Stony Man via Appalachian Trail,4.5,35.82,1,1.5,0hr 49mins,734,445,L,"[dogs-no, forest, kids, views, wild-flowers]","[birding, hiking, nature-trips, trail-running, walking]",trail,https://www.alltrails.com/trail/us/virginia/stony-man-via-appalachian-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.59318,-78.37599",skyland area,https://www.nps.gov/shen/planyourvisit/upload/Skyland_RoadTrail.pdf
0,Little Stony Man Loop via Appalachian Trail,4.5,34.4322,3,3.4,1hr 57mins,456,1047,L,"[beach, dogs-no, forest, kids, views, wild-flowers, wildlife]","[birding, hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/little-stony-man-loop-via-appalachian-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.59308,-78.37563",NaN,NaN
0,White Oak Canyon Trail,4.5,33.8864,5,9.5,5hr 27mins,445,1358,O,"[beach, dogs-leash, views, waterfall, wild-flowers, wildlife]","[backpacking, camping, hiking, nature-t

# Map District 

In [27]:
col_district = 'district'
col_skyline_drive_mile = 'skyline_drive_mile'
col_nps_features = 'nps_features'
col_nps_difficulty = 'nps_difficulty'

In [28]:
nps_df_html = pd.read_html('https://www.nps.gov/shen/planyourvisit/hiking-opportunities.htm')

In [29]:
df_north = nps_df_html[1]
df_central = nps_df_html[2]
df_south = nps_df_html[3]
for df, col in zip([df_north, df_central, df_south],['North','Central','South']):
    df.columns = [col_map_name, col_skyline_drive_mile, col_nps_features, col_nps_difficulty, col_length]
    df[col_district] = col

In [30]:
df_nps = pd.concat([df_north, df_central, df_south], axis=0, sort=False)

In [31]:
def map_trail_name(value):
    split_str = value.lower().split(' ')
    try:
        abbr_value = split_str[0] + split_str[1]
    except IndexError:
        abbr_value = split_str[0]
    abbr_value_0 = split_str[0]
#     print(abbr_value)
    for name in df_nps[col_map_name].values:
        clean_name = name.lower().replace(' ','')
        if clean_name.find(abbr_value.replace('trail','').replace('loop','')) != -1:
            map_name = name
#             print(name)
            break
        elif clean_name.find(abbr_value_0.replace('trail','').replace('loop','')) != -1:
            map_name = name
            break
        else:
            map_name = np.nan
    return map_name

In [32]:
df_final[col_map_name] = df_final[col_name].map(lambda x: map_trail_name(x))

In [33]:
df_final.reset_index(drop=True, inplace=True)
df_final.loc[df_final[col_map_name].isna(),[col_name, col_map_name]]

,name,nps_name
22,Mary's Rock via the Appalachian Trail (Southern Approach),NaN
24,Berry Hollow to Old Rag Mountain Trail,NaN
26,Hazel Falls and Caves,NaN
28,Buck Ridge Trail To Mary's Rock Loop,NaN
29,Appalachian Trail: Jones Run Trail to Pinefield Gap,NaN
34,Calvary and Chimney Rocks Trail,NaN
35,North Fork Moormans River,NaN
36,Shenandoah National Park Loop,NaN
39,Staunton River Trail to Bear Church Rock,NaN
40,Trayfoot Mountain Trail,NaN


In [34]:
filename = save_path + 'shenandoah_trail.xlsx'

In [36]:
# merge info on nps website
df_final_merge = df_final.merge(df_nps[[col_map_name, col_skyline_drive_mile, col_nps_features, col_nps_difficulty, col_district]],how='left', on=col_map_name)
df_final_merge.drop(col_map_name, axis=1, inplace=True)
df_final_merge.drop_duplicates(subset=col_name, keep='first', inplace=True)
df_final_merge.index = list(range(1, len(df_final_merge)+1))

In [37]:
# add heatmap
df_final_merge[col_difficulty]

,name,avg_rating,popularity,difficulty_rating,length,duration_minutes,num_reviews,num_photos,route_type,features,activities,type,link,direction,area_map,map_pdf,skyline_drive_mile,nps_features,nps_difficulty,ditrict
1,Old Rag Mountain Loop,5,76.8127,5,10.2,5hr 5mins,3062,2360,L,"[dogs-no, forest, river, views, wild-flowers, wildlife]",[hiking],trail,https://www.alltrails.com/trail/us/virginia/old-rag-mountain-loop-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.5714,-78.29314",old rag area,https://www.nps.gov/shen/planyourvisit/upload/OldRag_RoadTrail.pdf,See Old Rag page,"Viewpoint, rock scramble, no pets allowed",very strenuous,Central
2,White Oak Canyon and Cedar Run Trails Loop,4.5,45.0866,5,9.0,5hr 14mins,782,1793,L,"[dogs-leash, forest, river, views, waterfall, wild-flowers, wildlife]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/white-oak-canyon-and-cedar-run-trails?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53906,-78.34808",whiteoak canyon area,https://www.nps.gov/shen/planyourvisit/upload/WhiteoakCanyon_RoadTrail.pdf,42.6,"Waterfalls, stream, gorge, geology: columnar jointing",moderate to strenuous,Central
3,Dark Hollow Falls Trail,4.5,42.3311,3,1.4,0hr 57mins,1076,846,O,"[dogs-no, forest, kids, river, views, waterfall, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/dark-hollow-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.51971,-78.43083",big meadows area,https://www.nps.gov/shen/planyourvisit/upload/BigMeadows_RoadTrail.pdf,50.7,"Waterfall, no pets allowed","moderate, steep sections",Central
4,Hawksbill Gap Loop via Appalachian Trail,4.5,41.4116,3,2.6,1hr 33mins,708,1070,L,"[dogs, forest, kids, views, wild-flowers, wildlife]","[hiking, nature-trips, walking]",trail,https://www.alltrails.com/trail/us/virginia/hawksbill-gap-loop?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.55647,-78.38683",hawksbill area,https://www.nps.gov/shen/planyourvisit/upload/Hawksbill_RoadTrail.pdf,45.6,Viewpoint/Summit (highest peak in the park),strenuous,Central
5,Rose River Trail,4.5,40.3936,3,3.5,1hr 59mins,781,1258,L,"[beach, dogs-leash, forest, kids, river, waterfall, wild-flowers, wildlife]","[hiking, nature-trips, trail-running]",trail,https://www.alltrails.com/trail/us/virginia/rose-river-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.53317,-78.42117",big meadows area,https://www.nps.gov/shen/planyourvisit/upload/BigMeadows_RoadTrail.pdf,"49.4, Fishers Gap Overlook","Waterfall, stream, cascades",moderate,Central
6,Bearfence Mountain Trail,4.5,37.3653,3,1.0,0hr 34mins,748,540,L,"[dogs-no, forest, views, wild-flowers, wildlife]","[birding, hiking, nature-trips, fly-fishing]",trail,https://www.alltrails.com/trail/us/virginia/bearfence-mountain-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.45239,-78.4669",bearfence area,https://www.nps.gov/shen/planyourvisit/upload/Bearfence_RoadTrail.pdf,56.4,"Short, challenging hike with rock scramble to a view, no pets allowed",strenuous,Central
7,South River Falls Trail,4.5,36.0717,3,4.6,2hr 43mins,696,718,O,"[dogs-leash, forest, kids, river, views, waterfall]","[hiking, nature-trips]",trail,https://www.alltrails.com/trail/us/virginia/south-river-falls-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.38115,-78.51698",south river area,https://www.nps.gov/shen/planyourvisit/upload/SouthRiver_RoadTrail.pdf,62.8,"Waterfall, various approaches to South River Falls",moderate to strenuous,Central
8,Stony Man via Appalachian Trail,4.5,35.82,1,1.5,0hr 49mins,734,445,L,"[dogs-no, forest, kids, views, wild-flowers]","[birding, hiking, nature-trips, trail-running, walking]",trail,https://www.alltrails.com/trail/us/virginia/stony-man-via-appalachian-trail?ref=result-card,"https://www.google.com/maps/dir/Current+Location/38.59318,-78.37599",skylan

In [36]:
df_final_merge.to_excel(filename)